In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from yellowbrick.classifier import ConfusionMatrix

# Carregando dataset CIC train pre-processamento full

In [2]:
df_cic_train50 = pd.read_csv('/media/kelson/DataCenter/UFU/artigo/cic2017/df_cic_train50_with_preprocessing_getdummies.csv', low_memory=False)

In [3]:
df_cic_train50.head()

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,...,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label_multiclass,Label_binary,Destination_Port_Reserved,Destination_Port_User,Destination_Port_Private
0,48,2,0,0,0,0,0,0.0,0.000000,0,...,0,0.0,0.0,0,0,DoS,1,1,0,0
1,3,2,0,0,0,0,0,0.0,0.000000,0,...,0,0.0,0.0,0,0,BENIGN,0,0,0,1
2,84793047,10,6,334,11595,334,0,33.4,105.620074,4344,...,2975,83700000.0,0.0,83700000,83700000,DoS,1,1,0,0
3,25,1,1,2,6,2,2,2.0,0.000000,6,...,0,0.0,0.0,0,0,PortScan,1,0,1,0
4,49099,2,2,60,234,30,30,30.0,0.000000,117,...,0,0.0,0.0,0,0,BENIGN,0,1,0,0


# Divisão entre variáveis previsoras (x) e classe (y)

In [4]:
# atribuição dos valores para X e y
#X = df_cic_train50.iloc[:,:78]
X = df_cic_train50.drop(columns=['Label_multiclass', 'Label_binary'])
y_multiclass = df_cic_train50.Label_multiclass
y_binary = df_cic_train50.Label_binary
print('Dimensões para X: ', X.shape)
print('Dimensões para y_multiclass: ', y_multiclass.shape)
print('Dimensões para y_binary: ', y_binary.shape)

Dimensões para X:  (1415365, 80)
Dimensões para y_multiclass:  (1415365,)
Dimensões para y_binary:  (1415365,)


In [5]:
X

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Destination_Port_Reserved,Destination_Port_User,Destination_Port_Private
0,48,2,0,0,0,0,0,0.000000,0.000000,0,...,0.000000,0,0,0.0,0.000000e+00,0,0,1,0,0
1,3,2,0,0,0,0,0,0.000000,0.000000,0,...,0.000000,0,0,0.0,0.000000e+00,0,0,0,0,1
2,84793047,10,6,334,11595,334,0,33.400000,105.620074,4344,...,0.000000,2975,2975,83700000.0,0.000000e+00,83700000,83700000,1,0,0
3,25,1,1,2,6,2,2,2.000000,0.000000,6,...,0.000000,0,0,0.0,0.000000e+00,0,0,0,1,0
4,49099,2,2,60,234,30,30,30.000000,0.000000,117,...,0.000000,0,0,0.0,0.000000e+00,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415360,29171092,3,3,162,284,68,46,54.000000,12.165525,122,...,21152.392252,60507,30593,14509787.0,6.199332e+06,18893377,10126197,1,0,0
1415361,60,1,1,2,6,2,2,2.000000,0.000000,6,...,0.000000,0,0,0.0,0.000000e+00,0,0,0,0,1
1415362,26,1,1,0,6,0,0,0.000000,0.000000,6,...,0.000000,0,0,0.0,0.000000e+00,0,0,0,1,0
1415363,98437773,6,7,331,11595,331,0,55.166667,135.130184,4344,...,0.000000,4,4,98300000.0,0.000000e+00,98300000,98300000,1,0,0


In [6]:
y_multiclass

0                 DoS
1              BENIGN
2                 DoS
3            PortScan
4              BENIGN
              ...    
1415360        BENIGN
1415361      PortScan
1415362      PortScan
1415363           DoS
1415364    BruteForce
Name: Label_multiclass, Length: 1415365, dtype: object

In [7]:
y_binary

0          1
1          0
2          1
3          1
4          0
          ..
1415360    0
1415361    1
1415362    1
1415363    1
1415364    1
Name: Label_binary, Length: 1415365, dtype: int64

# Divisão das variáveis entre treino e teste

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train50_train, X_train50_test, y_train50_multiclass_train, y_train50_multiclass_test, y_train50_binary_train, y_train50_binary_test = train_test_split(X, y_multiclass, y_binary, test_size = 0.3, random_state=42)

In [10]:
print('X treino = ', X_train50_train.shape)
print('X teste = ', X_train50_test.shape)
print('y multiclasse treino = ', y_train50_multiclass_train.shape)
print('y multiclasse teste = ', y_train50_multiclass_test.shape)
print('y binário treino = ', y_train50_binary_train.shape)
print('y binário teste = ', y_train50_binary_test.shape)

X treino =  (990755, 80)
X teste =  (424610, 80)
y multiclasse treino =  (990755,)
y multiclasse teste =  (424610,)
y binário treino =  (990755,)
y binário teste =  (424610,)


# Pré-processamento com Padronização (StandadScaler)

In [11]:
from sklearn.preprocessing import StandardScaler
# a padronização e a normalização devem ser aplicadas somente nos dados de treino (X_train)
# esse método evita o vazamento de informação, evitando que o algoritmo já conheça os valores médios normalizados

In [12]:
X_train50_train.describe()

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Destination_Port_Reserved,Destination_Port_User,Destination_Port_Private
count,9.907550e+05,990755.000000,990755.000000,9.907550e+05,9.907550e+05,990755.000000,990755.000000,990755.000000,990755.000000,990755.000000,...,9.907550e+05,9.907550e+05,9.907550e+05,9.907550e+05,9.907550e+05,9.907550e+05,9.907550e+05,990755.000000,990755.000000,990755.000000
mean,1.568093e+07,11.436632,13.038048,5.432706e+02,2.155056e+04,197.552583,16.991644,52.396285,63.637149,940.188430,...,4.776135e+04,1.729220e+05,6.485094e+04,8.619872e+06,4.590788e+05,8.987230e+06,8.249796e+06,0.797549,0.086500,0.115951
std,3.442152e+07,927.455615,1234.662297,7.306866e+03,2.753819e+06,545.231135,44.012388,133.917399,191.818910,1988.834405,...,4.227884e+05,1.086646e+06,6.194933e+05,2.396346e+07,4.060421e+06,2.456960e+07,2.375058e+07,0.401827,0.281101,0.320166
min,-4.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,1.710000e+02,1.000000,1.000000,1.200000e+01,0.000000e+00,6.000000,0.000000,6.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000,0.000000
50%,4.713000e+04,2.000000,2.000000,6.400000e+01,1.360000e+02,40.000000,0.000000,36.000000,0.000000,90.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000,0.000000
75%,4.815506e+06,5.000000,5.000000,3.290000e+02,8.660000e+02,201.000000,37.000000,52.000000,73.076404,654.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000,0.000000
max,1.200000e+08,219759.000000,291922.000000,2.866110e+06,6.554530e+08,24820.000000,2065.000000,5940.857143,7049.469004,17376.000000,...,7.000000e+07,1.070000e+08,1.070000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08,1.000000,1.000000,1.000000


In [13]:
X_train50_train.head()

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Destination_Port_Reserved,Destination_Port_User,Destination_Port_Private
1008718,53396,2,2,66,386,33,33,33.000000,0.000000,193,...,0.0,0,0,0.0,0.0,0,0,1,0,0
1092907,47298,3,4,436,859,436,0,145.333333,251.724717,859,...,0.0,0,0,0.0,0.0,0,0,1,0,0
1034897,98166935,6,7,309,11595,291,0,51.500000,117.367372,4344,...,0.0,13035,13035,98100000.0,0.0,98100000,98100000,1,0,0
727692,60586,1,1,42,233,42,42,42.000000,0.000000,233,...,0.0,0,0,0.0,0.0,0,0,1,0,0
1200733,5294518,10,5,354,5458,197,0,35.400000,69.213358,2304,...,0.0,104930,104930,5189532.0,0.0,5189532,5189532,1,0,0


In [14]:
# mostrando valores máximos das variáveis 
X_train50_train['Flow_Duration'].max(), X_train50_train['Total_Fwd_Packets'].max(), X_train50_train['Total_Backward_Packets'].max(), X_train50_train['Total_Length_of_Fwd_Packets'].max(), X_train50_train['Total_Length_of_Bwd_Packets'].max()

(119999979, 219759, 291922, 2866110, 655453030)

In [15]:
# mostrando valores mínimos das variáveis 
X_train50_train['Flow_Duration'].min(), X_train50_train['Total_Fwd_Packets'].min(), X_train50_train['Total_Backward_Packets'].min(), X_train50_train['Total_Length_of_Fwd_Packets'].min(), X_train50_train['Total_Length_of_Bwd_Packets'].min()

(-4, 1, 0, 0, 0)

In [16]:
type(X_train50_train)

pandas.core.frame.DataFrame

In [17]:
# realizando a padronização
# deve ser aplicado o transform nos dados de test
scaler_cic = StandardScaler()
X_train50_train = scaler_cic.fit_transform(X_train50_train)

In [18]:
# O X_train50_train foi convertido para o formato numpy array durante a aplicação do scaler
type(X_train50_train)

numpy.ndarray

In [19]:
# mostrando valores máximos das variáveis 
X_train50_train[:,0].max(), X_train50_train[:,1].max(), X_train50_train[:,2].max(), X_train50_train[:,3].max(), X_train50_train[:,4].max()

(3.0306365364676124,
 236.93605466933818,
 236.4283010647289,
 392.17473154374625,
 238.00828055230195)

In [20]:
# mostrando valores mínimos das variáveis 
X_train50_train[:,0].min(), X_train50_train[:,1].min(), X_train50_train[:,2].min(), X_train50_train[:,3].min(), X_train50_train[:,4].min()

(-0.45555632132287344,
 -0.011252977241618134,
 -0.01056001658346923,
 -0.07435073857902716,
 -0.00782570179459418)

In [21]:
# atribuindo novo dataframe somente para visualizar os novos valores do X_train50_train
df_X_train50_train = pd.DataFrame(scaler_cic.transform(X_train50_train))
#df_X_train = pd.DataFrame(scaler_cic.transform(X_train))

/home/kelson/.local/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [22]:
df_X_train50_train.describe()

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
count,9.907550e+05,990755.000000,990755.000000,990755.000000,9.907550e+05,990755.000000,990755.000000,990755.000000,990755.000000,990755.000000,...,990755.000000,9.907550e+05,990755.000000,9.907550e+05,9.907550e+05,9.907550e+05,9.907550e+05,990755.000000,990755.000000,990755.000000
mean,-4.555562e-01,-0.012331,-0.010560,-0.074351,-7.825702e-03,-0.362328,-0.386065,-0.391258,-0.331757,-0.472734,...,-0.112968,-1.591338e-01,-0.104684,-3.597091e-01,-1.130619e-01,-3.657868e-01,-3.473516e-01,-1.984811,-0.307718,-0.362159
std,2.905163e-08,0.001078,0.000810,0.000137,3.631324e-07,0.001834,0.022721,0.007467,0.005213,0.000503,...,0.000002,9.202642e-07,0.000002,4.173024e-08,2.462801e-07,4.070075e-08,4.210428e-08,2.488639,3.557448,3.123380
min,-4.555562e-01,-0.012343,-0.010569,-0.074361,-7.825705e-03,-0.362993,-0.394837,-0.394180,-0.333486,-0.472971,...,-0.112968,-1.591340e-01,-0.104684,-3.597091e-01,-1.130619e-01,-3.657868e-01,-3.473516e-01,-6.924286,-1.402408,-1.493318
25%,-4.555562e-01,-0.012343,-0.010568,-0.074361,-7.825705e-03,-0.362973,-0.394837,-0.393846,-0.333486,-0.472971,...,-0.112968,-1.591340e-01,-0.104684,-3.597091e-01,-1.130619e-01,-3.657868e-01,-3.473516e-01,-0.730970,-1.402408,-1.493318
50%,-4.555562e-01,-0.012342,-0.010567,-0.074360,-7.825705e-03,-0.362858,-0.394837,-0.392173,-0.333486,-0.472949,...,-0.112968,-1.591340e-01,-0.104684,-3.597091e-01,-1.130619e-01,-3.657868e-01,-3.473516e-01,-0.730970,-1.402408,-1.493318
75%,-4.555562e-01,-0.012339,-0.010565,-0.074355,-7.825705e-03,-0.362317,-0.375736,-0.391281,-0.331500,-0.472806,...,-0.112968,-1.591340e-01,-0.104684,-3.597091e-01,-1.130619e-01,-3.657868e-01,-3.473516e-01,-0.730970,-1.402408,-1.493318
max,-4.555561e-01,0.243138,0.180932,-0.020679,-7.739273e-03,-0.279502,0.671196,-0.062915,-0.141896,-0.468578,...,-0.112576,-1.590434e-01,-0.104405,-3.597089e-01,-1.130573e-01,-3.657866e-01,-3.473514e-01,-0.730970,11.253017,8.262176


### Aplicando o transform nos dados de teste (X_train50_test)

In [23]:
X_train50_test.describe()

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Destination_Port_Reserved,Destination_Port_User,Destination_Port_Private
count,4.246100e+05,424610.000000,424610.000000,4.246100e+05,4.246100e+05,424610.000000,424610.000000,424610.000000,424610.000000,424610.000000,...,4.246100e+05,4.246100e+05,4.246100e+05,4.246100e+05,4.246100e+05,4.246100e+05,4.246100e+05,424610.000000,424610.000000,424610.000000
mean,1.575912e+07,10.591868,11.811714,5.603033e+02,1.952635e+04,199.790351,16.998450,52.841453,64.353076,939.989864,...,4.793019e+04,1.716019e+05,6.271150e+04,8.652013e+06,4.625414e+05,9.021304e+06,8.278383e+06,0.796246,0.087332,0.116422
std,3.455681e+07,828.605146,1087.670562,7.842047e+03,2.581977e+06,557.973337,43.345895,136.233017,194.638258,1988.447429,...,4.379139e+05,1.089199e+06,5.749990e+05,2.402339e+07,4.089085e+06,2.463550e+07,2.380775e+07,0.402789,0.282321,0.320731
min,-1.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,1.700000e+02,1.000000,1.000000,1.200000e+01,0.000000e+00,6.000000,0.000000,6.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000,0.000000
50%,4.679250e+04,2.000000,2.000000,6.400000e+01,1.380000e+02,40.000000,0.000000,36.000000,0.000000,91.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000,0.000000
75%,4.715760e+06,5.000000,5.000000,3.300000e+02,8.670000e+02,202.000000,37.000000,52.000000,73.283695,654.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000,0.000000
max,1.200000e+08,217797.000000,289585.000000,2.428415e+06,6.396506e+08,24820.000000,2293.000000,5939.285714,7125.596846,19530.000000,...,6.390000e+07,1.020000e+08,1.020000e+08,1.200000e+08,7.590000e+07,1.200000e+08,1.200000e+08,1.000000,1.000000,1.000000


In [24]:
# O X_train50_train foi convertido para o formato numpy array durante a aplicação do scaler
type(X_train50_test)

pandas.core.frame.DataFrame

In [25]:
# mostrando valores máximos das variáveis 
X_train50_test['Flow_Duration'].max(), X_train50_test['Total_Fwd_Packets'].max(), X_train50_test['Total_Backward_Packets'].max(), X_train50_test['Total_Length_of_Fwd_Packets'].max(), X_train50_test['Total_Length_of_Bwd_Packets'].max()

(119999987, 217797, 289585, 2428415, 639650620)

In [26]:
# mostrando valores mínimo das variáveis 
X_train50_test['Flow_Duration'].min(), X_train50_test['Total_Fwd_Packets'].min(), X_train50_test['Total_Backward_Packets'].min(), X_train50_test['Total_Length_of_Fwd_Packets'].min(), X_train50_test['Total_Length_of_Bwd_Packets'].min()

(-1, 1, 0, 0, 0)

In [27]:
# realizando a padronização
# deve ser aplicado somente o transform nos dados de test
X_train50_test = scaler_cic.transform(X_train50_test)

In [28]:
# O X_train50_train foi convertido para o formato numpy array durante a aplicação do scaler
type(X_train50_test)

numpy.ndarray

In [29]:
# mostrando valores máximos das variáveis 
X_train50_test[:,0].max(), X_train50_test[:,1].max(), X_train50_test[:,2].max(), X_train50_test[:,3].max(), X_train50_test[:,4].max()

(3.0306367688805027,
 234.82058848561522,
 234.5354748359862,
 332.2728219626198,
 232.26991651173782)

In [30]:
# mostrando valores máximos das variáveis 
X_train50_test[:,0].min(), X_train50_test[:,1].min(), X_train50_test[:,2].min(), X_train50_test[:,3].min(), X_train50_test[:,4].min()

(-0.45555623416803964,
 -0.011252977241618134,
 -0.01056001658346923,
 -0.07435073857902716,
 -0.00782570179459418)

# Salvando as variáveis X_train50_train, X_train50_test, y_train50_multiclass_train, y_train50_multiclass_test, y_train50_binary_train, y_train50_binary_test em arquivo único

In [31]:
import pickle

In [32]:
with open('df_cic_train50_with_preprocessing_getdummies_standadscaler', mode = 'wb') as f:
    pickle.dump([X_train50_train, X_train50_test, y_train50_multiclass_train, y_train50_multiclass_test, y_train50_binary_train, y_train50_binary_test], f)

# Carregando dataset CIC Test50 pre-processamento full

In [36]:
df_cic_test50 = pd.read_csv('/media/kelson/DataCenter/UFU/artigo/cic2017/df_cic_test50_with_preprocessing_getdummies.csv', low_memory=False)

In [37]:
df_cic_test50.head()

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,...,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label_multiclass,Label_binary,Destination_Port_Reserved,Destination_Port_User,Destination_Port_Private
0,5888635,3,1,0,0,0,0,0.0,0.000000,0,...,0,0.0,0.0,0,0,BENIGN,0,1,0,0
1,17,2,0,0,0,0,0,0.0,0.000000,0,...,0,0.0,0.0,0,0,DoS,1,1,0,0
2,174,2,2,76,160,38,38,38.0,0.000000,80,...,0,0.0,0.0,0,0,BENIGN,0,1,0,0
3,403986,26,0,2340,0,90,90,90.0,0.000000,0,...,0,0.0,0.0,0,0,BENIGN,0,0,1,0
4,119529,20,25,176,440,43,0,8.8,12.680694,52,...,0,0.0,0.0,0,0,BENIGN,0,1,0,0


# Divisão entre variáveis previsoras (x) e classe (y) do Test50

In [38]:
# atribuição dos valores para X e y
#X_test50 = df_cic_test50.iloc[:,:78]
X_test50 = df_cic_test50.drop(columns=['Label_multiclass', 'Label_binary'])
y_test50_multiclass = df_cic_test50.Label_multiclass
y_test50_binary = df_cic_test50.Label_binary
print('Dimensões para X: ', X_test50.shape)
print('Dimensões para y_multiclass: ', y_test50_multiclass.shape)
print('Dimensões para y_binary: ', y_test50_binary.shape)

Dimensões para X:  (1415365, 80)
Dimensões para y_multiclass:  (1415365,)
Dimensões para y_binary:  (1415365,)


In [39]:
X_test50

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Destination_Port_Reserved,Destination_Port_User,Destination_Port_Private
0,5888635,3,1,0,0,0,0,0.000000,0.000000,0,...,0.0,0,0,0.0,0.0,0,0,1,0,0
1,17,2,0,0,0,0,0,0.000000,0.000000,0,...,0.0,0,0,0.0,0.0,0,0,1,0,0
2,174,2,2,76,160,38,38,38.000000,0.000000,80,...,0.0,0,0,0.0,0.0,0,0,1,0,0
3,403986,26,0,2340,0,90,90,90.000000,0.000000,0,...,0.0,0,0,0.0,0.0,0,0,0,1,0
4,119529,20,25,176,440,43,0,8.800000,12.680694,52,...,0.0,0,0,0.0,0.0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415360,47934215,2,2,89,265,45,44,44.500000,0.707107,205,...,0.0,30556,30556,47873099.0,0.0,47873099,47873099,1,0,0
1415361,725488,2,2,88,662,44,44,44.000000,0.000000,331,...,0.0,0,0,0.0,0.0,0,0,1,0,0
1415362,144,2,2,76,132,38,38,38.000000,0.000000,66,...,0.0,0,0,0.0,0.0,0,0,1,0,0
1415363,83080581,7,6,614,11595,307,0,87.714286,149.800661,4344,...,0.0,4,4,82700000.0,0.0,82700000,82700000,1,0,0


In [40]:
y_test50_multiclass

0          BENIGN
1             DoS
2          BENIGN
3          BENIGN
4          BENIGN
            ...  
1415360    BENIGN
1415361    BENIGN
1415362    BENIGN
1415363       DoS
1415364    BENIGN
Name: Label_multiclass, Length: 1415365, dtype: object

In [41]:
y_test50_binary

0          0
1          1
2          0
3          0
4          0
          ..
1415360    0
1415361    0
1415362    0
1415363    1
1415364    0
Name: Label_binary, Length: 1415365, dtype: int64

# Pré-processamento com Padronização (StandadScaler) - somente transform

In [42]:
from sklearn.preprocessing import StandardScaler
# a padronização e a normalização devem ser aplicadas somente nos dados de treino (X_train)
# esse método evita o vazamento de informação, evitando que o algoritmo já conheça os valores médios normalizados
# deve ser aplicado somente o transform nos dados de test

In [43]:
X_test50.describe()

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Destination_Port_Reserved,Destination_Port_User,Destination_Port_Private
count,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,...,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06,1.415365e+06
mean,1.570439e+07,1.118320e+01,1.267015e+01,5.483804e+02,2.094329e+04,1.982239e+02,1.699369e+01,5.252984e+01,6.385193e+01,9.401289e+02,...,4.781200e+04,1.725260e+05,6.420911e+04,8.629515e+06,4.601176e+05,8.997452e+06,8.258372e+06,7.971583e-01,8.674935e-02,1.160923e-01
std,3.446217e+07,8.989423e+02,1.192469e+03,7.471448e+03,2.703413e+06,5.490856e+02,4.381349e+01,1.346164e+02,1.926693e+02,1.988718e+03,...,4.273821e+05,1.087412e+06,6.064884e+05,2.398145e+07,4.069040e+06,2.458938e+07,2.376774e+07,4.021157e-01,2.814675e-01,3.203357e-01
min,-4.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.700000e+02,1.000000e+00,1.000000e+00,1.200000e+01,0.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
50%,4.703300e+04,2.000000e+00,2.000000e+00,6.400000e+01,1.370000e+02,4.000000e+01,0.000000e+00,3.600000e+01,0.000000e+00,9.100000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,4.786069e+06,5.000000e+00,5.000000e+00,3.290000e+02,8.660000e+02,2.020000e+02,3.700000e+01,5.200000e+01,7.315182e+01,6.540000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,1.200000e+08,2.197590e+05,2.919220e+05,2.866110e+06,6.554530e+08,2.482000e+04,2.293000e+03,5.940857e+03,7.125597e+03,1.953000e+04,...,7.000000e+07,1.070000e+08,1.070000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08,1.000000e+00,1.000000e+00,1.000000e+00


In [44]:
# O X_test50 foi convertido para o formato numpy array durante a aplicação do scaler
type(X_test50)

pandas.core.frame.DataFrame

In [45]:
# mostrando valores máximos das variáveis
X_test50['Flow_Duration'].max(),X_test50['Total_Fwd_Packets'].max(), X_test50['Total_Backward_Packets'].max(), X_test50['Total_Length_of_Fwd_Packets'].max(), X_test50['Total_Length_of_Bwd_Packets'].max()

(119999987, 219759, 291922, 2866110, 655453030)

In [46]:
# mostrando valores mínimos das variáveis
X_test50['Flow_Duration'].min(),X_test50['Total_Fwd_Packets'].min(), X_test50['Total_Backward_Packets'].min(), X_test50['Total_Length_of_Fwd_Packets'].min(), X_test50['Total_Length_of_Bwd_Packets'].min()

(-4, 1, 0, 0, 0)

In [47]:
# realizando a padronização
# deve ser aplicado somente o transform nos dados de test
X_test50 = scaler_cic.transform(X_test50)

In [48]:
# O X_test foi convertido para o formato numpy array durante a aplicação do scaler
type(X_test50)

numpy.ndarray

In [49]:
# mostrando valores máximos das variáveis
X_test50[:,0].max(), X_test50[:,1].max(), X_test50[:,2].max(), X_test50[:,3].max(), X_test50[:,4].max()

(3.0306367688805027,
 236.93605466933818,
 236.4283010647289,
 392.17473154374625,
 238.00828055230195)

In [50]:
# mostrando valores mínimos das variáveis
X_test50[:,0].min(), X_test50[:,1].min(), X_test50[:,2].min(), X_test50[:,3].min(), X_test50[:,4].min()

(-0.45555632132287344,
 -0.011252977241618134,
 -0.01056001658346923,
 -0.07435073857902716,
 -0.00782570179459418)

# Salvando as variáveis X_test50, y_test50_multiclass,  y_test50_binary em arquivo único

In [51]:
with open('df_cic_test50_with_preprocessing_getdummies_standadscaler', mode = 'wb') as f:
    pickle.dump([X_test50, y_test50_multiclass, y_test50_binary], f)